In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-mtxt32aw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-mtxt32aw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=3bd2250f0fb8df345929e03ac3d37b8fb57bea7cc1120cb6c8ca210560e30263
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8imymh8/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [14]:
%reload_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [7]:
%%cu

#include <cuda_runtime.h>
#include <cstdio>
#include <stdio.h>
#include <stdlib.h>
#define N 10

__global__ void bfs(int* graph, bool* visited, int* queue, int start, int numVertices) {
    int queueStart = 0;
    int queueEnd = 0;
    queue[queueEnd++] = start;
    visited[start] = true;

    while (queueStart < queueEnd) 
    {
        int currVertex = queue[queueStart++];
        for (int neighbor = 0; neighbor < numVertices; neighbor++) {
            if (graph[currVertex * numVertices + neighbor] && !visited[neighbor]) {
                visited[neighbor] = true;
                queue[queueEnd++] = neighbor;
            }
        }
    }
}
int main() {
    
    //time
     cudaEvent_t start, stop;
    float elapsedTime;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);
    int numVertices = N;
    int graph[N][N] = 

{{0, 1, 1, 1, 0, 1, 0, 0, 0, 0}, 
{1, 0, 1, 1, 0, 0, 0, 0, 1, 0},  
{1, 1, 0, 0, 1, 1, 0, 1, 0, 0},
{1, 1, 0, 0, 1, 1, 0, 0, 0, 0}, 
{0, 0, 1, 1, 0, 0, 1, 1, 0, 1},
{1, 0, 1, 1, 0, 0, 1, 1, 0, 0}, 
{0, 0, 0, 0, 1, 1, 0, 1, 1, 0},
{0, 0, 1, 0, 1, 1, 1, 0, 1, 1}, 
{0, 1, 0, 0, 0, 0, 1, 1, 0, 1},
{0, 0, 0, 0, 1, 0, 0, 1, 1, 0}}
;
    bool visited[N] = {false};
   int queue[N];

    int startVertex = 2;

    int* devGraph;
    bool* devVisited;
    int* devQueue;

    cudaMalloc((void**)&devGraph, numVertices*numVertices*sizeof(int));
    cudaMalloc((void**)&devVisited, numVertices*sizeof(bool));
    cudaMalloc((void**)&devQueue, numVertices*sizeof(int));

    cudaMemcpy(devGraph, graph, numVertices*numVertices*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(devVisited, visited, numVertices*sizeof(bool), cudaMemcpyHostToDevice);
    cudaMemcpy(devQueue, queue, numVertices*sizeof(int), cudaMemcpyHostToDevice);
    
    /* varying threads */
    bfs<<<1 , numVertices>>>(devGraph, devVisited, devQueue, startVertex, numVertices);

    cudaMemcpy(visited, devVisited, numVertices*sizeof(bool), cudaMemcpyDeviceToHost);
    cudaMemcpy(queue, devQueue, numVertices*sizeof(int), cudaMemcpyDeviceToHost);

    cudaError_t err = cudaMemcpy(queue, devQueue, numVertices*sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
      printf("CUDA error: %s\n", cudaGetErrorString(err));
      }

/*
    printf("\n\nOrder of traversal of cities within a particular distance :\n");
     for(int i=0;i<N;i++)
    {
        printf(" %d ",queue[i]);
    }

    */

    cudaFree(devGraph);
    cudaFree(devVisited);
    cudaFree(devQueue);

    //time 
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    cudaEventElapsedTime(&elapsedTime, start, stop);

    printf("\n\nTime taken for exploring %d cities in the graph : %f in milliseconds \n", N,elapsedTime);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}




Time taken for exploring 10 cities in the graph : 0.292768 in milliseconds 

